In [3]:
import os
if 'Conformal-Sparsemax/notebooks' in os.getcwd():
    os.chdir(os.path.dirname(os.getcwd()))

In [4]:
from conformal_sparsemax.classifier import CNN, get_data,train,evaluate

In [5]:
import torch
from torch import nn
from sklearn.metrics import f1_score

In [6]:
from entmax.losses import SparsemaxLoss, Entmax15Loss

In [8]:
loss = 'sparsemax' #sparsemax or softmax
dataset = 'CIFAR100' #CIFAR100 or MNIST

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if loss == 'sparsemax':
    criterion = SparsemaxLoss()
elif loss == 'entmax15':
    criterion = Entmax15Loss()
elif loss == 'softmax':
    criterion = torch.nn.NLLLoss()
data= get_data(0.2,16,dataset = dataset)

if dataset == 'CIFAR100':
    model = CNN_CIFAR(loss).to(device)
elif dataset == 'MNIST':
    model = CNN(loss,n_classes=10,input_size=28,channels=1).to(device)
else:
    raise Exception('Wrong dataset name')
    
model, train_history, val_history, f1_history = train(model,
                                            train_dataloader,
                                            dev_dataloader,
                                            criterion,
                                            epochs=350,
                                            patience=3)

_, predicted_labels, true_labels, test_loss = evaluate(
                                                    model,
                                                    test_dataloader,
                                                    criterion)

f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f'Test loss: {test_loss:.3f}')
print(f'Test f1: {f1:.3f}')

Files already downloaded and verified
Files already downloaded and verified


NameError: name 'CNN_CIFAR' is not defined

In [ ]:
results = {
    'train_history':train_history,
    'val_history':val_history,
    'f1_history':f1_history,
}

In [ ]:
import json
with open(f'results/{dataset}_{loss}_results.json', 'w') as f:
    json.dump(results, f)

In [ ]:
torch.save(model.state_dict(), f'models/{dataset}_{loss}.pth')